# Simulateur de traffic routier
  
**Objectif** : Simuler le traffic dans le cas suivant

<img src="Banque d'image interface\Cas_etude_feux.png" width="500"/>


## Partie 1 : Création de la matrice qui modélisera notre route

### Syntaxte

Il faut voir le schéma du haut comme une compilation de blocs de différentes catégories routes, feux rouges, intersection, cedez le passage, debut, fin etc ...

Ainsi, notre matrice retraduira cet assemblage de blocs afin d'éviter la création de trop grosses matrices (si on redessine toute la route). Chaque élement de la matrice sera donc un élement de la route et devra respecter la syntaxe suivante :  

Element = ["Type" = str, Caractéristique du type = int]

Détail pour chaque élements :

- route = ['Route', sens de circulation 0 = ouest, 1 = sud ,2 = est, 3 = nord ]
- intersection = ['Intersection', proba : [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]]
- Feux rouges = ['Feu',sens de circulation, cycles en secondes]
- priorité = ['Priorite',sens de circulation ,coté concerné par la priorité [west : 0, south : 1, east : 0, north : 0]
- Début = ['Debut',sens de circulation, débit = int]
- Fin = ['Fin']


Cependant ces élements ne permetent pas de représenter ni de traiter l'évolution du traffic. C'est pourquoi il faut joindre à ces blocs des blocs 'transport' permettant de modéliser l'évolution du traffic en temps réel. 

C'est blocs transport se modéliseront de le manière suivantes :  

Transport = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1]  

Les 0 sont des espaces vides et les 1 des emplacements occupés. La capacité de l'élement précédant le bloc transport définira la longeur du bloc transport.  

Enfin, à chaque itération la suite de 0 et de 1 se déplacera vers la droite et lorsqu'un 1 atteindra le bord droit on ajoutera 1 à l'ocuppation du bloc suivant.  

On ajoutera c'est bloc transport à la fin des élements

### Hypothèses

Les voitures ne se déplace pas dinstantanément il y a un effet de 'chaine', ainsi il faudra à minima un 0 devant un & pour qu'il puisse avancer.

### Retour à notre cas

En apliquant le découpage on a : 

<img src="Banque d'image interface\Decoupage_route.jpg" width="500"/>


In [ ]:
# Cas 1 : feux rouges

Bloc_01 = ['Fin']
Bloc_02 = ['Intersection',[0,0,[0.5,0.5,0,0],0],[0]]
Bloc_03 = ['Intersection',[0,[1,0,0,0],[1,0,0,0],0],[0]]
Bloc_04 = ['Route',0,[0,0,0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Bloc_12 = ['Intersection',[[0,0.5,0.5,0],0,0,[0,1,0,0]],[0]]
Bloc_13 = ['Intersection',[[0,0,1,0],[0,0,0.5,0.5],0,0],[0]]



In [2]:
res = [0]*23

print(res)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
